In [2]:
import pandas as pd
import numpy as np

In [3]:
# Last edited: 2020-09 by JL
# Goal: get Survey of Earned Doctorates data into a format which doesn't suck
# Problems: The SEDS data from the NSF is great, but each year is available from like 80 different spreadsheets
# with questions concatenated in a super useless way, and a bunch of blank space
# Also, a lot of questions which are asked the same way in subsequent years are coded differently
# Same with the field names - why change them???
# 
# So here, I read all of those spreadsheets as dataframes, and set them up the way I want them. 
# If you believe me and just want the .csvs, let me know. but this should work for everyone!
#
# As always, feel free to reach out at jablevine@gmail.com

In [17]:
url = "https://ncses.nsf.gov/pubs/nsf20301/assets/data-tables/tables/nsf20301-tab056.xlsx"
seds_2017 = pd.read_excel(url, header = 3)
seds_2017 = seds_2017[seds_2017.Characteristic != "Unknown"].reset_index(drop = True)

begin_male = seds_2017[seds_2017.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2017[seds_2017.Characteristic=="Female doctorate recipients (number)"].index[0]
    
questions = []



for i in range(0,len(seds_2017)) :
    if i < begin_male :
        questions += [seds_2017.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2017.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2017.Characteristic[i]+"_female"]
            
seds_2017["questions"] = questions


for i in range(58 , 70 , 2) :
    url = "https://ncses.nsf.gov/pubs/nsf20301/assets/data-tables/tables/nsf20301-tab0"+str(i)+".xlsx"
    df = pd.read_excel(url, header = 3)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2017 = seds_2017.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])
    #seds_2017 = pd.concat(seds_2017, df)

#seds_2017.tail()



In [18]:
td_seds_2017 = seds_2017.T


td_seds_2017.columns = td_seds_2017.iloc[list(seds_2017.columns).index("questions")]

td_seds_2017["year"] = ["2017"]*len(td_seds_2017)


td_seds_2017 = td_seds_2017.drop(["Characteristic" , "questions"])


#This line gives the index the major_year. I'd prefer to keep these columns separate for Stata purposes. 
#Same line repeats for each year
#td_seds_2017.index = td_seds_2017.index+"_2017"




#td_seds_2017.head()

In [19]:
#okie doke let's do 2016 now! 
#copy-pasting
#then replacing all the 17s with 16s
#Next, fix the urls



url = "https://nsf.gov/statistics/2018/nsf18304/data/tab56.xlsx"
seds_2016 = pd.read_excel(url, header = 3)
seds_2016 = seds_2016[seds_2016.Characteristic != "Unknown"].reset_index(drop = True)
seds_2016.dropna(subset = ["Characteristic"], inplace=True)

#These two are new, in 2016 too
unwanted = seds_2016.columns[seds_2016.columns.str.startswith('Unnamed:')]
seds_2016.drop(columns=unwanted, inplace=True)


#wish I didn't have to hard-code this to last five rows
#this is the main difference between 2016 and 1017:
#needed to drop bottom rows, which were read in from the bottom of the csv
seds_2016.drop(seds_2016.tail(6).index,inplace=True) # drop last n rows



begin_male = seds_2016[seds_2016.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2016[seds_2016.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []


for i in range(0,len(seds_2016)) :
    if i < begin_male :
        questions += [seds_2016.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2016.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2016.Characteristic[i]+"_female"]

seds_2016["questions"] = questions

        
    
for i in range(58 , 70 , 2) :
    url = "https://nsf.gov/statistics/2018/nsf18304/data/tab"+str(i)+".xlsx"
    df = pd.read_excel(url, header = 3)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    
    df.drop(df.tail(6).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2016 = seds_2016.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])
    #seds_2016 = pd.concat(seds_2016, df)


#seds_2016.head()



In [20]:
td_seds_2016 = seds_2016.T
td_seds_2016.columns = td_seds_2016.iloc[list(seds_2016.columns).index("questions")]
td_seds_2016["year"] = ["2016"]*len(td_seds_2016)

td_seds_2016 = td_seds_2016.drop(["Characteristic" , "questions"])

#td_seds_2016.index = td_seds_2016.index+"_2016"

#they reported percents 0-1 for some reason in 2015 and 2016; here i multiply by 100 to fix that 
ugly = td_seds_2016.columns.to_list()
ok_cols = ['year', 'field', 'Age at doctorate (median years)_all', 'Age at doctorate (median years)_male', 'Age at doctorate (median years)_female',\
           'All doctorate recipients (number)a_all', 'Female doctorate recipients (number)_male', "From bachelor's_all"\
           "From bachelor's_male", "From bachelor's_female", 'From doctoral program startc_all', 'From doctoral program startc_male', 'From doctoral program startc_femal',\
          'From graduate school start_all', 'From graduate school start_male', 'From graduate school start_female', 'Male doctorate recipients (number)_male',\
          'Time to doctorate (median years)_all', "From bachelor's_all", 'Time to doctorate (median years)_male', "From bachelor's_male",\
          'Time to doctorate (median years)_female', 'From doctoral program startc_female']
to_edit = [x for x in ugly if x not in ok_cols]


for colm in to_edit :
    td_seds_2016[colm] = td_seds_2016[colm]*100



#td_seds_2016.head()

In [21]:
#okie doke let's do 2015 now! 
#copy-pasting
#then replacing all the 16s with 15s
#Next, fix the urls
#This one was just copy paste and a fixed url from 2016. Absolutely no errors. 



url = "https://nsf.gov/statistics/2017/nsf17306/data/tab56.xlsx"
seds_2015 = pd.read_excel(url, header = 3)
seds_2015 = seds_2015[seds_2015.Characteristic != "Unknown"].reset_index(drop = True)
seds_2015.dropna(subset = ["Characteristic"], inplace=True)
unwanted = seds_2015.columns[seds_2015.columns.str.startswith('Unnamed:')]
seds_2015.drop(columns=unwanted, inplace=True)


#wish I didn't have to hard-code this to last five rows
seds_2015.drop(seds_2015.tail(6).index,inplace=True) # drop last n rows



begin_male = seds_2015[seds_2015.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2015[seds_2015.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []



for i in range(0,len(seds_2015)) :
    if i < begin_male :
        questions += [seds_2015.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2015.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2015.Characteristic[i]+"_female"]

seds_2015["questions"] = questions


for i in range(58 , 70 , 2) :
    url = "https://nsf.gov/statistics/2017/nsf17306/data/tab"+str(i)+".xlsx"
    df = pd.read_excel(url, header = 3)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    
    df.drop(df.tail(6).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2015 = seds_2015.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])
    
    



#seds_2015.tail(10)

In [22]:
td_seds_2015 = seds_2015.T
td_seds_2015.columns = td_seds_2015.iloc[list(seds_2015.columns).index("questions")]
td_seds_2015["year"] = ["2015"]*len(td_seds_2015)

td_seds_2015 = td_seds_2015.drop(["Characteristic" , "questions"])


#THIS IS THE PLACE TO FIX THE UGLY PERCENTS
ugly = td_seds_2015.columns.to_list()
ok_cols = ['year', 'field', 'Age at doctorate (median years)_all', 'Age at doctorate (median years)_male', 'Age at doctorate (median years)_female',\
           'All doctorate recipients (number)a_all', 'Female doctorate recipients (number)_male', "From bachelor's_all"\
           "From bachelor's_male", "From bachelor's_female", 'From doctoral program startc_all', 'From doctoral program startc_male', 'From doctoral program startc_femal',\
          'From graduate school start_all', 'From graduate school start_male', 'From graduate school start_female', 'Male doctorate recipients (number)_male',\
          'Time to doctorate (median years)_all', "From bachelor's_all", 'Time to doctorate (median years)_male', "From bachelor's_male",\
          'Time to doctorate (median years)_female', 'From doctoral program startc_female']
to_edit = [x for x in ugly if x not in ok_cols]
print(to_edit)


for colm in to_edit :
    td_seds_2015[colm] = td_seds_2015[colm]*100

#td_seds_2015.index = td_seds_2015.index+"_2015"


#td_seds_2015.head()

['Sex (%)_all', 'Male_all', 'Female_all', 'Citizenship (%)_all', 'U.S. citizen or permanent resident_all', 'Temporary visa holder_all', 'Marital status (%)_all', 'Never married_all', 'Married_all', 'Marriage-like relationship_all', 'Separated, divorced, widowed_all', "Bachelor's in same field as doctorate (%)b_all", "Master's earned (%)_all", 'Citizenship (%)_male', 'U.S. citizen or permanent resident_male', 'Temporary visa holder_male', 'Marital status (%)_male', 'Never married_male', 'Married_male', 'Marriage-like relationship_male', 'Separated, divorced, widowed_male', "Bachelor's in same field as doctorate (%)b_male", "Master's earned (%)_male", 'Citizenship (%)_female', 'U.S. citizen or permanent resident_female', 'Temporary visa holder_female', 'Marital status (%)_female', 'Never married_female', 'Married_female', 'Marriage-like relationship_female', 'Separated, divorced, widowed_female', "Bachelor's in same field as doctorate (%)b_female", "Master's earned (%)_female"]


In [23]:
#okie doke let's do 2014 now! 
#copy-pasting
#then replacing all the 15s with 14s
#Next, fix the urls



url = "https://nsf.gov/statistics/2016/nsf16300/data/tab56.xlsx"
seds_2014 = pd.read_excel(url, header = 1)
seds_2014 = seds_2014[seds_2014.Characteristic != "Unknown"].reset_index(drop = True)
seds_2014.dropna(subset = ["Characteristic"], inplace=True)
unwanted = seds_2014.columns[seds_2014.columns.str.startswith('Unnamed:')]
seds_2014.drop(columns=unwanted, inplace=True)
seds_2014.reset_index(drop = True , inplace = True)


#wish I didn't have to hard-code this to last five rows
seds_2014.drop(seds_2014.tail(4).index,inplace=True) # drop last n rows



begin_male = seds_2014[seds_2014.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2014[seds_2014.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []



for i in range(0,len(seds_2014)) :
    if i < begin_male :
        questions += [seds_2014.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2014.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2014.Characteristic[i]+"_female"]

seds_2014["questions"] = questions


for i in range(58 , 70 , 2) :
    url = "https://nsf.gov/statistics/2016/nsf16300/data/tab"+str(i)+".xlsx"
    df = pd.read_excel(url, header = 1)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    df.reset_index(drop = True , inplace = True)

    
    df.drop(df.tail(4).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2014 = seds_2014.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])


#THIS WORKS
#seds_2014.head()


In [24]:
td_seds_2014 = seds_2014.T

td_seds_2014.index = td_seds_2014.index.str.replace('\n', '', regex=True)

td_seds_2014.columns = td_seds_2014.iloc[list(seds_2014.columns).index("questions")]
td_seds_2014["year"] = ["2014"]*len(td_seds_2014)


td_seds_2014 = td_seds_2014.drop(["Characteristic" , "questions"])

#td_seds_2014.index = td_seds_2014.index+"_2014"


#td_seds_2014.head()


In [25]:
#okie doke last one!! 2013 
#copy-pasting
#then replacing all the 14s with 13s
#Next, fix the urls




url = "https://nsf.gov/statistics/sed/2013/data/tab56.xlsx"
seds_2013 = pd.read_excel(url, header = 1)
seds_2013 = seds_2013[seds_2013.Characteristic != "Unknown"].reset_index(drop = True)
seds_2013.dropna(subset = ["Characteristic"], inplace=True)
unwanted = seds_2013.columns[seds_2013.columns.str.startswith('Unnamed:')]
seds_2013.drop(columns=unwanted, inplace=True)
seds_2013.reset_index(drop = True , inplace = True)


#wish I didn't have to hard-code this to last five rows
seds_2013.drop(seds_2013.tail(5).index,inplace=True) # drop last n rows



begin_male = seds_2013[seds_2013.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2013[seds_2013.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []




for i in range(0,len(seds_2013)) :
    if i < begin_male :
        questions += [seds_2013.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2013.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2013.Characteristic[i]+"_female"]

seds_2013["questions"] = questions


for i in range(58 , 70 , 2) :
    url = "https://nsf.gov/statistics/sed/2013/data/tab"+str(i)+".xlsx"
    df = pd.read_excel(url, header = 1)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    df.reset_index(drop = True , inplace = True)

    
    df.drop(df.tail(5).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2013 = seds_2013.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])




#seds_2013.tail()


In [26]:
td_seds_2013 = seds_2013.T

td_seds_2013.index = td_seds_2013.index.str.replace('\n', '', regex=True)

td_seds_2013.columns = td_seds_2013.iloc[list(seds_2013.columns).index("questions")]
td_seds_2013["year"] = ["2013"]*len(td_seds_2013)

td_seds_2013 = td_seds_2013.drop(["Characteristic" , "questions"])

#td_seds_2013.index = td_seds_2013.index+"_2013"


#td_seds_2013.head()

In [27]:
#merge 2013-2017 into one df

#seds_13_17 = pd.DataFrame()

#dfs = [td_seds_2013, td_seds_2014, td_seds_2015, td_seds_2016, td_seds_2017]

#for i in dfs:  
#    seds_13_17 = seds_13_17.append(i)
    
#seds_13_17.head()


In [28]:
folder = "C:/Users/240-370-8956/Dropbox/Research/Visas/Data/NSF Doctorates/SED_2012/data"
    
seds_2012 = pd.read_excel(folder+"/tab56.xlsx", header = 1)
seds_2012 = seds_2012[seds_2012.Characteristic != "Unknown"].reset_index(drop = True)
seds_2012.dropna(subset = ["Characteristic"], inplace=True)
unwanted = seds_2012.columns[seds_2012.columns.str.startswith('Unnamed:')]
seds_2012.drop(columns=unwanted, inplace=True)
seds_2012.reset_index(drop = True , inplace = True)

#wish I didn't have to hard-code this to last five rows
seds_2012.drop(seds_2012.tail(5).index,inplace=True) # drop last n rows


begin_male = seds_2012[seds_2012.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2012[seds_2012.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []




for i in range(0,len(seds_2012)) :
    if i < begin_male :
        questions += [seds_2012.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2012.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2012.Characteristic[i]+"_female"]

seds_2012["questions"] = questions

for i in range(58 , 70 , 2) :
    df = pd.read_excel(folder+"/tab"+str(i)+".xlsx", header = 1)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    df.reset_index(drop = True , inplace = True)

    
    df.drop(df.tail(5).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2012 = seds_2012.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])


td_seds_2012 = seds_2012.T

td_seds_2012.index = td_seds_2012.index.str.replace('\n', '', regex=True)

td_seds_2012.columns = td_seds_2012.iloc[list(seds_2012.columns).index("questions")]
td_seds_2012["year"] = ["2012"]*len(td_seds_2012)

td_seds_2012 = td_seds_2012.drop(["Characteristic" , "questions"])

#td_seds_2012.index = td_seds_2012.index+"_2012"


#td_seds_2012.head()





In [29]:
folder = "C:/Users/240-370-8956/Dropbox/Research/Visas/Data/NSF Doctorates/sed_2011/data"
    
seds_2011 = pd.read_excel(folder+"/tab56.xls", header = 1)
seds_2011 = seds_2011[seds_2011.Characteristic != "Unknown"].reset_index(drop = True)
seds_2011.dropna(subset = ["Characteristic"], inplace=True)
unwanted = seds_2011.columns[seds_2011.columns.str.startswith('Unnamed:')]
seds_2011.drop(columns=unwanted, inplace=True)
seds_2011.reset_index(drop = True , inplace = True)

#wish I didn't have to hard-code this to last five rows
seds_2011.drop(seds_2011.tail(5).index,inplace=True) # drop last n rows


begin_male = seds_2011[seds_2011.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2011[seds_2011.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []




for i in range(0,len(seds_2011)) :
    if i < begin_male :
        questions += [seds_2011.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2011.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2011.Characteristic[i]+"_female"]

seds_2011["questions"] = questions

for i in range(58 , 70 , 2) :
    df = pd.read_excel(folder+"/tab"+str(i)+".xls", header = 1)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    df.reset_index(drop = True , inplace = True)

    
    df.drop(df.tail(5).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2011 = seds_2011.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])


td_seds_2011 = seds_2011.T

td_seds_2011.index = td_seds_2011.index.str.replace('\n', '', regex=True)

td_seds_2011.columns = td_seds_2011.iloc[list(seds_2011.columns).index("questions")]
td_seds_2011["year"] = ["2011"]*len(td_seds_2011)

td_seds_2011 = td_seds_2011.drop(["Characteristic" , "questions"])

#td_seds_2011.index = td_seds_2011.index+"_2011"


#td_seds_2011.head()





In [30]:
folder = "C:/Users/240-370-8956/Dropbox/Research/Visas/Data/NSF Doctorates/sed_2010/data"
    
seds_2010 = pd.read_excel(folder+"/tab56.xls", header = 1)
seds_2010 = seds_2010[seds_2010.Characteristic != "Unknown"].reset_index(drop = True)
seds_2010.dropna(subset = ["Characteristic"], inplace=True)
unwanted = seds_2010.columns[seds_2010.columns.str.startswith('Unnamed:')]
seds_2010.drop(columns=unwanted, inplace=True)
seds_2010.reset_index(drop = True , inplace = True)

#wish I didn't have to hard-code this to last five rows
seds_2010.drop(seds_2010.tail(5).index,inplace=True) # drop last n rows


begin_male = seds_2010[seds_2010.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2010[seds_2010.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []




for i in range(0,len(seds_2010)) :
    if i < begin_male :
        questions += [seds_2010.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2010.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2010.Characteristic[i]+"_female"]

seds_2010["questions"] = questions

for i in range(58 , 70 , 2) :
    df = pd.read_excel(folder+"/tab"+str(i)+".xls", header = 1)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    df.reset_index(drop = True , inplace = True)

    
    df.drop(df.tail(5).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2010 = seds_2010.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])


td_seds_2010 = seds_2010.T

td_seds_2010.index = td_seds_2010.index.str.replace('\n', '', regex=True)

td_seds_2010.columns = td_seds_2010.iloc[list(seds_2010.columns).index("questions")]
td_seds_2010["year"] = ["2010"]*len(td_seds_2010)

td_seds_2010 = td_seds_2010.drop(["Characteristic" , "questions"])

#td_seds_2010.index = td_seds_2010.index+"_2010"

#these lines fix inconsistent column/question names/ punctuation for 2009 and 2010
td_seds_2010 = td_seds_2010.rename(columns={"Age at doctorate_all": "Age at doctorate (median years)_all", "Age at doctorate_female": "Age at doctorate (median years)_female", "Age at doctorate_male": "Age at doctorate (median years)_male"})
td_seds_2010 = td_seds_2010.rename(columns={"All doctorate recipients (number)_all": "All doctorate recipients (number)a_all"})
td_seds_2010 = td_seds_2010.rename(columns={"Bachelor's in same field as doctoratea_all": "Bachelor's in same field as doctorate (%)b_all", "Bachelor's in same field as doctoratea_female": "Bachelor's in same field as doctorate (%)b_female", "Bachelor's in same field as doctoratea_male": "Bachelor's in same field as doctorate (%)b_male"})
td_seds_2010 = td_seds_2010.rename(columns={"Master's earned_all": "Master's earned (%)_all", "Master's earned_female": "Master's earned (%)_female", "Master's earned_male": "Master's earned (%)_male"})



#td_seds_2010.head()





In [31]:
folder = "C:/Users/240-370-8956/Dropbox/Research/Visas/Data/NSF Doctorates/sed_2009/data"
    
seds_2009 = pd.read_excel(folder+"/tab52.xls", header = 1)
seds_2009 = seds_2009[seds_2009.Characteristic != "Unknown"].reset_index(drop = True)
seds_2009.dropna(subset = ["Characteristic"], inplace=True)
unwanted = seds_2009.columns[seds_2009.columns.str.startswith('Unnamed:')]
seds_2009.drop(columns=unwanted, inplace=True)
seds_2009.reset_index(drop = True , inplace = True)

#wish I didn't have to hard-code this to last five rows
seds_2009.drop(seds_2009.tail(5).index,inplace=True) # drop last n rows


begin_male = seds_2009[seds_2009.Characteristic=="Male doctorate recipients (number)"].index[0]
begin_female = seds_2009[seds_2009.Characteristic=="Female doctorate recipients (number)"].index[0]


    
questions = []




for i in range(0,len(seds_2009)) :
    if i < begin_male :
        questions += [seds_2009.Characteristic[i]+"_all"]
    elif begin_male <= i <= begin_female :
        questions += [seds_2009.Characteristic[i]+"_male"]
    elif i > begin_female :
        questions += [seds_2009.Characteristic[i]+"_female"]

seds_2009["questions"] = questions

for i in range(54 , 66 , 2) :
    df = pd.read_excel(folder+"/tab"+str(i)+".xls", header = 1)
    df = df[df.Characteristic != "Unknown"].reset_index(drop = True)
    df.dropna(subset = ["Characteristic"], inplace=True)
    unwanted = df.columns[df.columns.str.startswith('Unnamed:')]
    df.drop(columns=unwanted, inplace=True)
    df.reset_index(drop = True , inplace = True)

    
    df.drop(df.tail(5).index,inplace=True) # drop last n rows

    
    begin_male = df[df.Characteristic=="Male doctorate recipients (number)"].index[0]
    begin_female = df[df.Characteristic=="Female doctorate recipients (number)"].index[0]
    
    questions = []
    

    
    for i in range(0,len(df)) :
        if i < begin_male :
            questions += [df.Characteristic[i]+"_all"]
        elif begin_male <= i <= begin_female :
            questions += [df.Characteristic[i]+"_male"]
        elif i > begin_female :
            questions += [df.Characteristic[i]+"_female"]
            
    df["questions"] = questions
    
    seds_2009 = seds_2009.join(df.set_index(["questions","Characteristic"]) , on=["questions","Characteristic"])


td_seds_2009 = seds_2009.T

td_seds_2009.index = td_seds_2009.index.str.replace('\n', '', regex=True)

td_seds_2009.columns = td_seds_2009.iloc[list(seds_2009.columns).index("questions")]
td_seds_2009["year"] = ["2009"]*len(td_seds_2009)

td_seds_2009 = td_seds_2009.drop(["Characteristic" , "questions"])

#td_seds_2009.index = td_seds_2009.index+"_2009"


td_seds_2009 = td_seds_2009.rename(columns={"Age at doctorate_all": "Age at doctorate (median years)_all", "Age at doctorate_female": "Age at doctorate (median years)_female", "Age at doctorate_male": "Age at doctorate (median years)_male"})
td_seds_2009 = td_seds_2009.rename(columns={"All doctorate recipients (number)_all": "All doctorate recipients (number)a_all"})
td_seds_2009 = td_seds_2009.rename(columns={"Bachelor's in same field as doctoratea_all": "Bachelor's in same field as doctorate (%)b_all", "Bachelor's in same field as doctoratea_female": "Bachelor's in same field as doctorate (%)b_female", "Bachelor's in same field as doctoratea_male": "Bachelor's in same field as doctorate (%)b_male"})
td_seds_2009 = td_seds_2009.rename(columns={"Master's earned_all": "Master's earned (%)_all", "Master's earned_female": "Master's earned (%)_female", "Master's earned_male": "Master's earned (%)_male"})

#td_seds_2009.head()


In [33]:
#appned 2009-2017 into one df

#create blank dataframe to append into 
seds_09_17 = pd.DataFrame()

#list of the dataframes I want to append (I know, I know)
dfs = [td_seds_2009, td_seds_2010, td_seds_2011, td_seds_2012, td_seds_2013, td_seds_2014, td_seds_2015, td_seds_2016, td_seds_2017]


for i in dfs:  
    seds_09_17 = seds_09_17.append(i)
    
#name the index (first column, in Stata)
seds_09_17.index.set_names('field', inplace=True)
    
#because a lot of rows in the original excels are empty, now a lot of columns are
#so i'm dropping everythign with an entirely empty column
seds_09_17 = seds_09_17.dropna(axis= 'columns', how='all')

seds_09_17.to_csv("C:/Users/240-370-8956/Dropbox/Research/Visas/Analysis/seds_09_17.csv")





In [ ]:
#and we can't go further back than 2009! So that's all the data we got. 
#See the next Stata .do file for my analysis there